In [1]:
import SimpleITK as sitk
import numpy as np
import nibabel as nib
from scipy.ndimage import affine_transform
from scipy import ndimage
from pathlib import Path

# Provide the file paths for the fixed (anchor) and moving (distorted) images
fixed_image_path = 'data/segthor_train/train/Patient_27/GT2.nii.gz'
moving_image_path = 'data/segthor_train/train/Patient_27/GT.nii.gz'



In [2]:
replace_index = 2

def register_images(fixed_img_file, moving_img_file, output_transform_file, replace_index=None):
    """
    Register two images and save the transform.
    """
    # Read the images
    fixed_img = sitk.ReadImage(fixed_img_file, sitk.sitkFloat32)
    moving_img = sitk.ReadImage(moving_img_file, sitk.sitkFloat32)
    
    if replace_index:
        fixed_img = sitk.GetArrayFromImage(fixed_img)
        fixed_img = (fixed_img == replace_index).astype(np.float32)
        fixed_img = sitk.GetImageFromArray(fixed_img)
        
        moving_img = sitk.GetArrayFromImage(moving_img)
        moving_img = (moving_img == replace_index).astype(np.float32)
        moving_img = sitk.GetImageFromArray(moving_img)

    # Setup the registration
    registration = sitk.ImageRegistrationMethod()
    registration.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration.SetMetricSamplingStrategy(registration.RANDOM)
    registration.SetMetricSamplingPercentage(0.02)
    registration.SetInterpolator(sitk.sitkNearestNeighbor)
    registration.SetOptimizerAsGradientDescent(learningRate=0.8, numberOfIterations=200, convergenceMinimumValue=1e-7, convergenceWindowSize=1)
    registration.SetOptimizerScalesFromPhysicalShift()
    registration.SetInitialTransform(sitk.CenteredTransformInitializer(fixed_img, moving_img, sitk.AffineTransform(3), True))
    registration.SetShrinkFactorsPerLevel(shrinkFactors=[4,2,1])
    registration.SetSmoothingSigmasPerLevel(smoothingSigmas=[2,1,0])
    registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
    
    # Run the registration
    transform = registration.Execute(fixed_img, moving_img)
    
    # Save the transform
    sitk.WriteTransform(transform, output_transform_file)

In [3]:
# Step 2: Apply affine transformation using SciPy
def apply_affine_transform(moving_image, transform_map):
    # transform_parameters = list(map(float, transform_map.GetParameterMap(1).get('TransformParameters') + transform_map.GetParameterMap(0).get('TransformParameters')))

    # Hardcoded transform parameters from the raw transform file
    transform_parameters = (1,0,0,0,1,0,0,0,1,-66.58184178354128,-38.56866728429671,-14.999585668903961)
    affine_matrix = np.array([
        [transform_parameters[0], transform_parameters[1], transform_parameters[2], transform_parameters[9]],
        [transform_parameters[3], transform_parameters[4], transform_parameters[5], transform_parameters[10]],
        [transform_parameters[6], transform_parameters[7], transform_parameters[8], transform_parameters[11]],
        [0, 0, 0, 1]
    ])

    rotation_scale = affine_matrix[:3, :3]
    translation = affine_matrix[:3, 3]

    moving_image_data = np.array(moving_image)
    transformed_image_data = affine_transform(moving_image_data, rotation_scale, offset=translation, order=0)

    object_center = ndimage.center_of_mass(transformed_image_data)
    volume_center = np.array(transformed_image_data.shape) // 2
    # shift = volume_center - np.array(object_center)
    # Hardcoded shift
    shift = np.array([-30.68731694,  21.74786194,  20.2083591 ])

    shifted_volume = ndimage.shift(transformed_image_data, shift, order=0)

    angle = -21  # Rotation angle in degrees
    axes = (0,1)  # Rotation in the (x,y) plane

    # Rotate the 3D volume
    rotated_volume = ndimage.rotate(shifted_volume, angle, axes=axes, reshape=False, order=0)

    # Shift the rotated volume back to the original position
    transformed_image_data = ndimage.shift(rotated_volume, -shift, order=0)

    return transformed_image_data

# Step 3: Save the result using Nibabel
def save_image_with_nibabel(transformed_image_data, reference_image_path, output_image_path, replace_index=None):
    reference_image = nib.load(reference_image_path)
    reference_affine = reference_image.affine
    reference_header = reference_image.header
    
    if replace_index:
        reference_image_data = reference_image.get_fdata()
        reference_image_data[reference_image_data == replace_index] = 0
        reference_image_data[transformed_image_data == 1] = replace_index
        transformed_image_data = reference_image_data

    transformed_image_nifti = nib.Nifti1Image(transformed_image_data.astype(np.uint8), reference_affine, reference_header)
    nib.save(transformed_image_nifti, output_image_path)


In [4]:

# Load the fixed and moving images
fixed_image = nib.load(fixed_image_path).get_fdata()
fixed_image = (fixed_image == replace_index).astype(np.float32)

moving_image = nib.load(moving_image_path).get_fdata()
moving_image = (moving_image == replace_index).astype(np.float32)

output_image_path = 'transformed_image.nii.gz'

# Perform registration and get the transform map
print("Registering images...")
register_images(fixed_image_path, moving_image_path, 'transform.tfm')
# result_image, transform_params = register_images(fixed_image_path, moving_image_path, replace_index)

transform_params = None
# Apply the affine transformation
print("Applying affine transformation...")
transformed_image_data = apply_affine_transform(moving_image, transform_params)
# print(transform_params)

# Save the final transformed image
print("Saving the transformed image...")
save_image_with_nibabel(transformed_image_data.round(), moving_image_path, output_image_path, replace_index)

print(f"Final overlap ratio: {(transformed_image_data == fixed_image).sum() / fixed_image.size:.4f}")

Registering images...
Applying affine transformation...
Saving the transformed image...
Final overlap ratio: 0.9996


In [1]:
import os
base_folder = 'data/segthor_train/train'
transform_file = 'data/Slicer BSpline Transform.h5'
# Loop through all patient folders
for patient_id in range(1, 41):  # Assuming 40 patients
    patient_folder = f'Patient_{patient_id:02d}'
    input_seg_file = os.path.join(base_folder, patient_folder, 'GT.nii.gz')
    transformed_image_filename = 'transformed_GT_manual.nii.gz'
    transformed_image_path = os.path.join(base_folder, patient_folder, transformed_image_filename)
    
    # Check if the original GT file exists
    if os.path.exists(input_seg_file):
        # Rename the original GT file to GT_original.nii.gz
        original_gt_file = os.path.join(base_folder, patient_folder, 'GT_original.nii.gz')
        os.rename(input_seg_file, original_gt_file)
        print(f"Renamed {input_seg_file} to {original_gt_file}")

        new_gt_file = os.path.join(base_folder, patient_folder, 'GT.nii.gz')
        os.rename(transformed_image_path, new_gt_file)

Renamed data/segthor_train/train/Patient_01/GT.nii.gz to data/segthor_train/train/Patient_01/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_02/GT.nii.gz to data/segthor_train/train/Patient_02/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_03/GT.nii.gz to data/segthor_train/train/Patient_03/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_04/GT.nii.gz to data/segthor_train/train/Patient_04/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_05/GT.nii.gz to data/segthor_train/train/Patient_05/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_06/GT.nii.gz to data/segthor_train/train/Patient_06/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_07/GT.nii.gz to data/segthor_train/train/Patient_07/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_08/GT.nii.gz to data/segthor_train/train/Patient_08/GT_original.nii.gz
Renamed data/segthor_train/train/Patient_09/GT.nii.gz to data/segthor_train/train/Patient_09/GT_original